<h3>1-2</h3>


In [1]:
import torch
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

device = "cpu"  # "cuda" if torch.cuda.is_available else "cpu"
torch.set_default_device(device)

device

'cpu'

In [2]:
import os

classes = train_directories = os.listdir("./TRAIN/")
discretized_classes = [i for i in range(classes.__len__())]
classes, discretized_classes

(['disgust',
  'anger',
  'contempt',
  'happy',
  'fear',
  'neutral',
  'surprise',
  'sad'],
 [0, 1, 2, 3, 4, 5, 6, 7])

In [4]:
transform = transforms.Compose([transforms.PILToTensor()])

X = torch.zeros((8000 + 8000, 3, 128, 128))
finetune = torch.zeros((1600 + 1600, 3, 128, 128))

for process in ["TRAIN", "TUNE"]:
    for i in range(classes.__len__()):
        files = os.listdir(f"./{process}/{classes[i]}/")
        for j in range(files.__len__()):
            image = Image.open(f"./{process}/{classes[i]}/{files[j]}")

            augmented_image = transforms.RandomAffine(
                degrees=20, translate=(0.1, 0.1), shear=30
            )(image)

            if process == "TRAIN":
                X[i + j] = transform(image)
                X[i + j + 1] = transform(augmented_image)
            else:
                finetune[i + j] = transform(image)
                finetune[i + j + 1] = transform(augmented_image)
y = torch.zeros(8000)
y_finetune = torch.zeros(1600)
for i in range(discretized_classes.__len__()):
    y[i * 1000 : i * 1000 + 1000] = discretized_classes[i]
    y_finetune[i * 200 : i * 200 + 200] = discretized_classes[i]

X.shape, y.shape, finetune.shape

: 